# Imports and Setups

In [1]:
!pip install -q -U bitsandbytes
!pip install -q -U git+https://github.com/huggingface/transformers.git
!pip install -q -U git+https://github.com/huggingface/peft.git
!pip install -q -U git+https://github.com/huggingface/accelerate.git

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 92.6/92.6 MB 14.9 MB/s eta 0:00:00
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 268.8/268.8 kB 4.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.8/7.8 MB 105.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.3/1.3 MB 76.3 MB/s eta 0:00:00
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 251.2/251.2 kB 3.9 MB/s eta 0:00:00
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done


In [1]:
! nvidia-smi

Fri Sep  1 21:13:27 2023       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 525.105.17   Driver Version: 525.105.17   CUDA Version: 12.0     |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|                               |                      |               MIG M. |
|===============================+======================+======================|
|   0  Tesla V100-SXM2...  Off  | 00000000:00:04.0 Off |                    0 |
| N/A   34C    P0    25W / 300W |      0MiB / 16384MiB |      0%      Default |
|                               |                      |                  N/A |
+-------------------------------+----------------------+----------------------+
                                                                               
+-------

Lendo o Arquivo

In [2]:
from google.colab import drive
drive.mount('drive')

Mounted at drive


In [3]:
import pandas as pd
from tqdm import tqdm
tqdm.pandas()


In [4]:
num_gen = 5
perc = 5
i=2

In [5]:
arq = '/content/drive/My Drive/Colab Notebooks/t5_key_text/perc_' +str(perc) + '/emotion/base_emotions_'+str(perc)+'_original'+str(i)+'.csv'
base_cru_1p = pd.read_csv(arq, encoding='utf-8', sep=';', decimal = ',')

In [6]:
base_cru_1p.head()

,texto,label
0,i miss time with my husband and not feeling ru...,3
1,i feel tortured with tiredness everyday,4
2,i was feeling bouncy so i added a few of my go...,1
3,i feel like that s because i didn t grow up in...,1
4,i feel liked these days by both tom and myself,2


In [7]:
def cria_prompt(ordem,synt):
  prompt = ordem

  prompt += 'Sentence: '+ synt+'\nResult: '
  return prompt

In [8]:
inicio = """Rewrite the following sentence 5 times to generate 5 different paraphrases  from original tweet.
Return in Python list like this ["paraphrase1","paraphrase2","paraphrase3","paraphrase4","paraphrase5"] as example.

Sentence:What are the best places to see in New York?
Result:["What are some must-see places in New York?","Can you suggest some must-see spots in New York?","Where should one go to experience the best NYC has to offer?","Which places should I visit in New York?","What are the top destinations to explore in New York?"]
###
"""
base_cru_1p['prompt'] = base_cru_1p.progress_apply(lambda x: cria_prompt(inicio, x.texto), axis=1)
base_cru_1p.head()

100%|██████████| 800/800 [00:00<00:00, 21370.21it/s]


,texto,label,prompt
0,i miss time with my husband and not feeling ru...,3,Rewrite the following sentence 5 times to gene...
1,i feel tortured with tiredness everyday,4,Rewrite the following sentence 5 times to gene...
2,i was feeling bouncy so i added a few of my go...,1,Rewrite the following sentence 5 times to gene...
3,i feel like that s because i didn t grow up in...,1,Rewrite the following sentence 5 times to gene...
4,i feel liked these days by both tom and myself,2,Rewrite the following sentence 5 times to gene...


In [9]:
base_cru_1p.texto.iloc[90]

'i feel miserable just reading about americas heat wave and i live in the always hot middle east'

In [10]:
print(base_cru_1p.prompt.iloc[90])

Rewrite the following sentence 5 times to generate 5 different paraphrases  from original tweet.
Return in Python list like this ["paraphrase1","paraphrase2","paraphrase3","paraphrase4","paraphrase5"] as example.

Sentence:What are the best places to see in New York?
Result:["What are some must-see places in New York?","Can you suggest some must-see spots in New York?","Where should one go to experience the best NYC has to offer?","Which places should I visit in New York?","What are the top destinations to explore in New York?"]
###
Sentence: i feel miserable just reading about americas heat wave and i live in the always hot middle east
Result: 


In [11]:
n_labels = base_cru_1p.label.nunique()
n_labels

6

In [2]:
! huggingface-cli login


    _|    _|  _|    _|    _|_|_|    _|_|_|  _|_|_|  _|      _|    _|_|_|      _|_|_|_|    _|_|      _|_|_|  _|_|_|_|
    _|    _|  _|    _|  _|        _|          _|    _|_|    _|  _|            _|        _|    _|  _|        _|
    _|_|_|_|  _|    _|  _|  _|_|  _|  _|_|    _|    _|  _|  _|  _|  _|_|      _|_|_|    _|_|_|_|  _|        _|_|_|
    _|    _|  _|    _|  _|    _|  _|    _|    _|    _|    _|_|  _|    _|      _|        _|    _|  _|        _|
    _|    _|    _|_|      _|_|_|    _|_|_|  _|_|_|  _|      _|    _|_|_|      _|        _|    _|    _|_|_|  _|_|_|_|
    
    To login, `huggingface_hub` requires a token generated from https://huggingface.co/settings/tokens .
Token: 
Add token as git credential? (Y/n) y
Token is valid (permission: read).
Cannot authenticate through git-credential as no helper is defined on your machine.
You might have to re-authenticate when pushing to the Hugging Face Hub.
Run the following command in your terminal in case you want to set the 'store' cre

In [3]:
import torch
from transformers import AutoTokenizer, AutoModelForCausalLM, BitsAndBytesConfig

model_id = "meta-llama/Llama-2-7b-chat-hf"
bnb_config = BitsAndBytesConfig(
    load_in_4bit=True,
    bnb_4bit_use_double_quant=True,
    bnb_4bit_quant_type="nf4",
    bnb_4bit_compute_dtype=torch.bfloat16
)

tokenizer = AutoTokenizer.from_pretrained(model_id)
model_4bit = AutoModelForCausalLM.from_pretrained(model_id, quantization_config=bnb_config, device_map="auto")

Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]

In [4]:
prompt = """Rewrite the following tweets 5 times to generate 5 different paraphrases  from original tweet.
Return in Python list like this ["paraphrase1","paraphrase2","paraphrase3","paraphrase4","paraphrase5"] as example.

Tweet:What are the best places to see in New York?
Result:["What are some must-see places in New York?","Can you suggest some must-see spots in New York?","Where should one go to experience the best NYC has to offer?","Which places should I visit in New York?","What are the top destinations to explore in New York?"]
###
Tweet: The Quarterback' wrecks me every time..
Result: """

In [5]:
def gera_text(modelo, tokens, dev, ppt, temperatura, max_tokens):
  inputs = tokens(ppt, return_tensors="pt").to(dev)
  outputs = modelo.generate(**inputs, max_new_tokens=max_tokens, temperature=temperatura, do_sample =True)
  result = tokenizer.decode(outputs[0], skip_special_tokens=True)
  return result

In [7]:
device = "cuda"
print(gera_text(model_4bit, tokenizer, device, prompt, 0.1, 150))

Rewrite the following tweets 5 times to generate 5 different paraphrases  from original tweet.
Return in Python list like this ["paraphrase1","paraphrase2","paraphrase3","paraphrase4","paraphrase5"] as example.

Tweet:What are the best places to see in New York?
Result:["What are some must-see places in New York?","Can you suggest some must-see spots in New York?","Where should one go to experience the best NYC has to offer?","Which places should I visit in New York?","What are the top destinations to explore in New York?"]
###
Tweet: The Quarterback' wrecks me every time..
Result:  ["The Quarterback's impact never fails to move me...", "The Quarterback's brilliance always leaves me in awe...", "The Quarterback's talent is unmatched and always impresses me...", "The Quarterback's greatness never ceases to amaze me...", "The Quarterback's skills are unparalleled and always leave me in wonder..."]





























































In [18]:
base_cru_1p['generated'] = base_cru_1p['prompt'].progress_apply(lambda x: gera_text(model_4bit, tokenizer, device, x, 0.1, 150))
base_cru_1p.head()


  0%|          | 1/800 [00:20<4:30:38, 20.32s/it]


KeyboardInterrupt: ignored

In [ ]:
base_cru_1p.head()

In [ ]:
import ast
def transforme_list(x):
  try:
    resp = ast.literal_eval(x.split("Result:")[2].replace("\n", "").strip())
  except:
    resp = []
  return resp

In [ ]:
print(base_cru_1p.generated.iloc[4])

In [ ]:
base_cru_1p.generated.iloc[0].split("Result:")[2].replace("\n", "").strip()

In [ ]:
print(ast.literal_eval(base_cru_1p.generated.iloc[3].split("Result:")[2].replace("\n", "")))

In [ ]:
base_cru_1p['generated'] = base_cru_1p['generated'].progress_apply(lambda x: transforme_list(x))

In [ ]:
base_cru_1p.head()

In [ ]:
export = base_cru_1p.copy()
export = export.explode('generated').reset_index(drop=True)


In [ ]:
export.head()

In [ ]:
export.shape

In [ ]:
base_cru_1p.shape

In [ ]:
export.shape[0]/base_cru_1p.shape[0]

In [ ]:
arquivo = '/content/drive/My Drive/Colab Notebooks/t5_key_text/perc_' +str(perc) + '/emotion/gptparaphrase/base_emotions_'+str(perc)+'_gptparaphrase'+str(i)+'.pkl'
export[['texto', 'label', 'generated']].to_pickle(arquivo)